# CEIA - Procesamiento de Lenguaje Natural

## Estudiante: a2124 - Ricardo Silvera

--- 

# Desafío 2

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado**

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.
- Elegir términos de interés y buscar términos más similares y menos similares.
- Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.
- Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

# Importar plotly para visualizaciones interactivas
try:
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    # Configurar plotly para trabajar en notebooks
    pyo.init_notebook_mode(connected=True)
except:
    !pip install plotly nbformat
    import plotly.graph_objects as go
    import plotly.express as px
    from plotly.subplots import make_subplots
    import plotly.offline as pyo
    pyo.init_notebook_mode(connected=True)

In [2]:
# Instalar dependencias necesarias para plotly
!pip install -q plotly nbformat ipywidgets kaleido

## Carga de dataset
Leo el dataset propuesto en el desafio , que contine las letras de múltiples interpretes

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Leer el archivo de texto de Nirvana
with open('songs_dataset/bob-marley.txt', 'r', encoding='utf-8') as f:
    texto = f.read()

print(f"Texto cargado: {len(texto)} caracteres")



Texto cargado: 91527 caracteres


In [5]:
# Importar librerías adicionales para procesamiento de texto
import re
import string
from collections import Counter
import nltk

# Descargar datos necesarios de NLTK
try:
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
    nltk.download('stopwords', quiet=True)
    print("Datos de NLTK descargados correctamente")
except Exception as e:
    print(f"Error descargando NLTK: {e}")
    pass

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

Datos de NLTK descargados correctamente


## Preprocesamiento del texto

Vamos a procesar el texto de las letrade de Bob Marle para prepararlo para el entrenamiento del modelo Word2Vec. Para ello realizo los sigueintes pasos:
-Separo en oraciones con la funcion `sent_tokenize` que reconoce signo de puntuación para separar oraciones y retorna un lista de string.
- Luego limpio cada oración eliminando simbolos de puntución y números
- Realizo la tokenización por palabra con la función `word_tokenize` , con loq ue consigo una lista compuesta por listas de palabras.
- Elimino las palabras cortas (menores a 3 letras)
- Utilizo *stopwords* para eliminar las palabras comunies del idioma inglés que no sunam valor semántico.

In [6]:
def preprocess_text(texto):
    """
    Preprocesa el texto, separando el texto en oraciones
    """
    # Paso todo a minúsculas
    texto = texto.lower()

    stop_words = set(stopwords.words())
    
    # Divido en oraciones
    oraciones = sent_tokenize(texto)
    
    
    # Tokenizar cada oración y limpiar
    oraciones_limpias = []
    
    for oracion in oraciones:
        # Eliminar puntuación y números
        oracion = re.sub(r'[^\w\s]', '', oracion)
        oracion = re.sub(r'\d+', '', oracion)

        # Tokenizar en palabras
        tokens = word_tokenize(oracion)
        
        # Elimino palabras muy cortas 
        tokens = [token for token in tokens if len(token) > 2 
                  and token.isalpha()
                  and token not in stop_words 
                  ]
        
        # Solo mantengo oraciones con al menos 3 palabras
        if len(tokens) > 1:  
            oraciones_limpias.append(tokens)
    
    return oraciones_limpias



In [7]:
# Procesar el texto
oraciones_procesadas = preprocess_text(texto)

print(f"Número de oraciones: {len(oraciones_procesadas)}")
print(f"Ejemplo de oraciones procesadas:")
for i in range(5):
    print(f"{i+1}: {oraciones_procesadas[i]}")
    print("-"*100)

Número de oraciones: 433
Ejemplo de oraciones procesadas:
1: ['worry', 'gon']
----------------------------------------------------------------------------------------------------
2: ['singin', 'worry', 'gon']
----------------------------------------------------------------------------------------------------
3: ['rise', 'mornin', 'smiled', 'risin', 'sun', 'birds', 'perch', 'doorstep', 'singin', 'sweet', 'songs', 'melodies', 'pure', 'sayin', 'message', 'yououou', 'singin', 'worry', 'bout', 'gon']
----------------------------------------------------------------------------------------------------
4: ['singin', 'worry', 'worry', 'bout', 'gon']
----------------------------------------------------------------------------------------------------
5: ['rise', 'mornin', 'smiled', 'risin', 'sun', 'birds', 'perch', 'doorstep', 'singin', 'sweet', 'songs', 'melodies', 'pure', 'sayin', 'message', 'yououou', 'singin', 'worry', 'worry']
-----------------------------------------------------------------

## Entrenamiento del modelo

Entreno el modelo Word2Vec

In [16]:

modelo_w2v = Word2Vec(
    min_count=3,          # frecuencia mínima de palabra para incluirla en el vocabulario
    window=3,             # Ventana de contexto
    vector_size=200,      # Dimensión de los vectores
    sentences=oraciones_procesadas,
    workers=4,
    sg=1,                     # Skip-gram (1) vs CBOW (0)
)

modelo_w2v.build_vocab(oraciones_procesadas)

print(f"Cantidad de docuentos en el corpus:{modelo_w2v.corpus_count}")
print("Cantidad de palabraas distintas en el corpus:", len(modelo_w2v.wv.index_to_key))

# Mostrar algunas palabras del vocabulario
print("\nAlgunas palabras en el vocabulario:")
palabras_vocab = list(modelo_w2v.wv.key_to_index.keys())[:20]
print(palabras_vocab)

Cantidad de docuentos en el corpus:433
Cantidad de palabraas distintas en el corpus: 515

Algunas palabras en el vocabulario:
['love', 'gon', 'jah', 'baby', 'jammin', 'give', 'coming', 'wan', 'feel', 'lord', 'stand', 'worry', 'easy', 'dread', 'time', 'simmer', 'life', 'woman', 'ooh', 'stir']


## Exploración de términos similares

Vamos a explorar los embeddings encontrando palabras similares y no similares a términos de interés relacionados con las letras de Bob Marley.

In [17]:
# Definir términos de interés 
terminos_interes = ['woman', 'love', 'fight', 'cry', 'alright', 'baby','rastaman', 'children']

# Filtrar términos que existen en el vocabulario
terminos_disponibles = [termino for termino in terminos_interes 
                       if termino in modelo_w2v.wv.key_to_index]

print(f"Términos disponibles en el vocabulario: {terminos_disponibles}")

# Explorar similitudes para cada término disponible
for termino in terminos_disponibles: 
    print(f"\n=== Términos similares a '{termino}' ===")
    try:
        similares = modelo_w2v.wv.most_similar(termino, topn=8)
        for palabra, similaridad in similares:
            print(f"  {palabra}: {similaridad:.3f}")
    except KeyError:
        print(f"  '{termino}' no está en el vocabulario")
    
    # Mostrar términos menos similares (opuestos)
    print(f"\n--- Términos menos similares a '{termino}' ---")
    try:
        # Para encontrar términos opuestos, buscamos los de menor similitud
        todas_palabras = list(modelo_w2v.wv.key_to_index.keys())[:500]  # Muestra para eficiencia
        similitudes = [(palabra, modelo_w2v.wv.similarity(termino, palabra)) 
                      for palabra in todas_palabras[:100] if palabra != termino]
        menos_similares = sorted(similitudes, key=lambda x: x[1])[:5]
        
        for palabra, similaridad in menos_similares:
            print(f"  {palabra}: {similaridad:.3f}")
    except:
        print("  No se pudo calcular")

Términos disponibles en el vocabulario: ['woman', 'love', 'fight', 'cry', 'alright', 'baby', 'rastaman', 'children']

=== Términos similares a 'woman' ===
  sister: 0.997
  listen: 0.996
  free: 0.996
  smile: 0.996
  call: 0.996
  hear: 0.996
  school: 0.996
  put: 0.996

--- Términos menos similares a 'woman' ---
  hit: 0.599
  easy: 0.951
  movement: 0.963
  cold: 0.973
  wall: 0.978

=== Términos similares a 'love' ===
  land: 0.997
  worry: 0.996
  bring: 0.996
  smiled: 0.996
  sayin: 0.996
  put: 0.996
  listen: 0.996
  hear: 0.996

--- Términos menos similares a 'love' ---
  hit: 0.586
  easy: 0.954
  movement: 0.965
  cold: 0.974
  refuse: 0.978

=== Términos similares a 'fight' ===
  note: 0.997
  listen: 0.997
  pass: 0.997
  bring: 0.997
  put: 0.997
  everythings: 0.997
  sayin: 0.997
  bob: 0.997

--- Términos menos similares a 'fight' ---
  hit: 0.595
  easy: 0.955
  movement: 0.962
  cold: 0.976
  music: 0.978

=== Términos similares a 'cry' ===
  generation: 0.997
  br

Vemos que el indice de similaridad es muy alto para las palabras elegidas, incluso con las palabras menos similares. Esto se debe a que es un conteto muy específico, con un vocabulario reducido hace que los terminos estén vinculados de alguna manera.

## Reducción de dimensionalidad y visualización

Ahora vamos a reducir los embeddings a 2 dimensiones usando PCA y visualizar los términos más frecuentes.

In [18]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import numpy as np

# Configurar el número máximo de palabras para visualización
MAX_WORDS = 150

# Obtener las palabras más frecuentes del vocabulario
contador_palabras = Counter()
for oracion in oraciones_procesadas:
    contador_palabras.update(oracion)

# Seleccionar las MAX_WORDS palabras más frecuentes que están en el modelo
palabras_frecuentes = []
vectores_frecuentes = []

for palabra, freq in contador_palabras.most_common():
    if palabra in modelo_w2v.wv.key_to_index and len(palabras_frecuentes) < MAX_WORDS:
        palabras_frecuentes.append(palabra)
        vectores_frecuentes.append(modelo_w2v.wv[palabra])

vectores_frecuentes = np.array(vectores_frecuentes)

print(f"Seleccionadas {len(palabras_frecuentes)} palabras para visualización")
print(f"Dimensión original de los vectores: {vectores_frecuentes.shape}")

# Reducción de dimensionalidad con PCA
pca = PCA(n_components=2)
vectores_2d_pca = pca.fit_transform(vectores_frecuentes)

print(f"Varianza explicada por PCA: {pca.explained_variance_ratio_}")
print(f"Varianza total explicada: {sum(pca.explained_variance_ratio_):.3f}")

# También probamos con t-SNE para comparación
tsne = TSNE(n_components=2, perplexity=30, random_state=42)
vectores_2d_tsne = tsne.fit_transform(vectores_frecuentes)

Seleccionadas 150 palabras para visualización
Dimensión original de los vectores: (150, 200)
Varianza explicada por PCA: [0.963022   0.00324924]
Varianza total explicada: 0.966


In [23]:
# Crear visualización interactiva con Plotly - Embeddings con PCA
import plotly.graph_objects as go
import plotly.io as pio

# Configurar renderer para VS Code
pio.renderers.default = "vscode"

# Crear figura
fig = go.Figure()

# Gráfico PCA con todas las palabras
fig.add_trace(
    go.Scatter(
        x=vectores_2d_pca[:, 0],
        y=vectores_2d_pca[:, 1],
        mode='markers+text',
        marker=dict(size=8, opacity=0.6, color='steelblue'),
        text=[palabras_frecuentes[i] if i % 10 == 0 else '' for i in range(len(palabras_frecuentes))],
        textposition='top center',
        textfont=dict(size=8),
        hovertext=palabras_frecuentes,
        hoverinfo='text',
        name='Palabras'
    )
)

# Actualizar layout
fig.update_layout(
    title=f'Embeddings proyectados con PCA - Bob Marley (Top {MAX_WORDS} palabras)',
    xaxis_title='Componente Principal 1',
    yaxis_title='Componente Principal 2',
    height=700,
    width=1200,
    hovermode='closest',
    showlegend=False
)

# Añadir anotación con información de varianza explicada
fig.add_annotation(
    text=f"Varianza PCA explicada: {sum(pca.explained_variance_ratio_):.1%}",
    xref="paper", yref="paper",
    x=0.02, y=0.98,
    showarrow=False,
    font=dict(size=12),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="black",
    borderwidth=1
)

# Mostrar el gráfico
fig.show()

En el gráfico de embeddings proyectados con PCA se observa una **concentración importante de palabras en el centro del espacio bidimensional**, lo que indica que la mayoría del vocabulario comparte contextos similares en las letras de Bob Marley.

Al trabajar con un corpus reducido, el modelo Word2Vec captura principalmente **relaciones de co-ocurrencia** más que relaciones semánticas profundas. Las palabras cercanas en el gráfico tienden a aparecer juntas en versos o estrofas similares.

El PCA explica aproximadamente un **25% de la varianza** en solo 2 componentes, lo que confirma que el espacio semántico es complejo y multidimensional. Muchas relaciones importantes entre palabras se pierden al reducir de 200 a 2 dimensiones.

La cercanía espacial en este gráfico refleja más la **frecuencia de aparición conjunta** en las letras que una verdadera similitud semántica.

Para obtener representaciones vectoriales con mayor riqueza semántica, sería necesario entrenar el modelo con un corpus más amplio y diverso, o utilizar embeddings pre-entrenados en grandes cantidades de texto en inglés.

## Análisis de grupos de palabras

Vamos a identificar grupos de palabras similares en el espacio de embeddings para encontrar patrones semánticos.

In [24]:
# Análisis de grupos usando clustering
from sklearn.cluster import KMeans
import plotly.io as pio

# Configurar renderer para VS Code
pio.renderers.default = "vscode"

# Aplicar K-means para encontrar grupos de palabras similares
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(vectores_frecuentes)

# Crear DataFrame para facilitar la visualización
df_clusters = pd.DataFrame({
    'palabra': palabras_frecuentes,
    'x': vectores_2d_pca[:, 0],
    'y': vectores_2d_pca[:, 1],
    'cluster': clusters
})

# Visualizar los clusters con Plotly
fig = go.Figure()

# Colores para cada cluster
colors = px.colors.qualitative.Set3[:n_clusters]

# Añadir puntos por cluster
for i in range(n_clusters):
    cluster_data = df_clusters[df_clusters['cluster'] == i]
    
    # Seleccionar las 3 palabras más representativas del cluster para etiquetar
    palabras_mostrar = cluster_data.head(3)['palabra'].tolist()
    
    fig.add_trace(go.Scatter(
        x=cluster_data['x'],
        y=cluster_data['y'],
        mode='markers+text',
        marker=dict(size=10, opacity=0.7, color=colors[i],
                   line=dict(width=1, color='white')),
        text=[palabra if palabra in palabras_mostrar else '' 
              for palabra in cluster_data['palabra']],
        textposition='top center',
        textfont=dict(size=9),
        hovertext=cluster_data['palabra'],
        hoverinfo='text',
        name=f'Cluster {i+1}'
    ))

fig.update_layout(
    title=dict(
        text='Clustering de palabras en espacio 2D (PCA)<br><sub>K-means con 8 clusters</sub>',
        x=0.5,
        xanchor='center'
    ),
    xaxis_title='Componente Principal 1',
    yaxis_title='Componente Principal 2',
    height=700,
    width=1200,
    hovermode='closest',
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.01
    )
)

# Mostrar el gráfico
fig.show()

# Mostrar el contenido de cada cluster
print("\n=== ANÁLISIS DE CLUSTERS ===")
for i in range(n_clusters):
    cluster_words = [palabras_frecuentes[j] for j in range(len(palabras_frecuentes)) if clusters[j] == i]
    print(f"\nCluster {i+1} ({len(cluster_words)} palabras):")
    print(f"  Primeras 15 palabras: {cluster_words[:15]}")


=== ANÁLISIS DE CLUSTERS ===

Cluster 1 (31 palabras):
  Primeras 15 palabras: ['jah', 'baby', 'stand', 'time', 'simmer', 'ooh', 'pon', 'soul', 'away', 'ganja', 'loving', 'africa', 'rights', 'talk', 'wait']

Cluster 2 (20 palabras):
  Primeras 15 palabras: ['give', 'jammin', 'woman', 'heart', 'sweet', 'hear', 'lets', 'ive', 'make', 'things', 'place', 'great', 'whoa', 'everythings', 'sayin']

Cluster 3 (28 palabras):
  Primeras 15 palabras: ['easy', 'stir', 'hammer', 'movement', 'youths', 'heathen', 'wall', 'bomb', 'rat', 'skanking', 'music', 'jamaica', 'hit', 'natty', 'pimpers']

Cluster 4 (1 palabras):
  Primeras 15 palabras: ['smile']

Cluster 5 (22 palabras):
  Primeras 15 palabras: ['wan', 'day', 'back', 'yoy', 'children', 'rebel', 'world', 'long', 'woe', 'unite', 'trouble', 'chuck', 'kingston', 'dey', 'night']

Cluster 6 (3 palabras):
  Primeras 15 palabras: ['worry', 'youre', 'singin']

Cluster 7 (23 palabras):
  Primeras 15 palabras: ['love', 'gon', 'coming', 'feel', 'lord', 'l


El algoritmo K-means identificó **8 clusters** que agrupan palabras con contextos de uso similares en las letras de Bob Marley. Alguno de estos grupos comparten una temática específica:

#### **Cluster 1 - Espiritualidad**
Palabras clave: `jah`, `soul`, `africa`, `rights`, `stand`

Este cluster agrupa términos de de la cultura rastafari y de lucha social. Incluye referencias espirituales ("jah" - Dios en el rastafarianismo), África como tierra ancestral, y términos relacionados con derechos y resistencia. Existen sietas canciones de Bob Marley que tranta esta temática y utiliza estos términos, que genera contexto similares para ests términos. 

#### **Cluster 2  - Amor**
Palabras clave: `give`, `woman`, `heart`, `sweet`, `loving`

Agrupa vocabulario romántico de Bob Marley. Términos como "woman", "heart", "sweet" y "loving" reflejan las canciones de amor que son parte  de su repertorio. 

#### **Cluster 3  - Cultura Jamaiquina**
Palabras clave: `music`, `jamaica`, `skanking`, `natty`, `hammer`

Representa el contexto musical y cultural jamaiquino. 

#### **Cluster 7  - Emociones**
Palabras clave: `love`, `feel`, `life`, `cry`, `fight`, `happy`

Agrupa el espectro emocional presente en las canciones. 



## Análisis semántico específico

Exploremos relaciones semánticas específicas entre conceptos temáticos presentes en las letras de Bob Marley.

In [25]:
# Análisis de relaciones semánticas entre conceptos temáticos de Bob Marley
print("=== ANÁLISIS SEMÁNTICO DE CONCEPTOS CLAVE ===\n")

# Buscar palabras relacionadas con los temas principales de Bob Marley
conceptos_buscar = {
    'amor_romance': ['love', 'woman', 'baby', 'heart', 'sweet'],
    'espiritualidad': ['jah', 'lord', 'soul', 'pray', 'god'],
    'lucha_social': ['fight', 'freedom', 'rights', 'rebel', 'stand'],
    'emociones': ['feel', 'happy', 'cry', 'worry', 'smile'],
    'musica': ['jammin', 'music', 'rock', 'singing', 'song'],
    'rastafari': ['rastaman', 'dread', 'natty', 'ganja', 'africa']
}

# Encontrar qué conceptos están disponibles en nuestro vocabulario
conceptos_disponibles = {}
for categoria, palabras in conceptos_buscar.items():
    palabras_encontradas = [p for p in palabras if p in modelo_w2v.wv.key_to_index]
    if palabras_encontradas:
        conceptos_disponibles[categoria] = palabras_encontradas

for categoria, palabras in conceptos_disponibles.items():
    print(f"\n--- {categoria.upper().replace('_', ' ')} ---")
    print(f"Palabras disponibles: {palabras}")
    
    # Para cada palabra, mostrar las más similares
    for palabra in palabras[:2]:  # Limitamos a 2 por categoría
        try:
            similares = modelo_w2v.wv.most_similar(palabra, topn=5)
            print(f"  Similares a '{palabra}': {[s[0] for s in similares]}")
        except:
            continue

# Análisis de relaciones entre conceptos clave
print(f"\n=== SIMILITUD ENTRE CONCEPTOS PRINCIPALES ===")
conceptos_principales = [
    ('love', 'jah', 'Amor y Espiritualidad'),
    ('fight', 'freedom', 'Lucha y Libertad'),
    ('music', 'jammin', 'Música y Celebración'),
    ('woman', 'baby', 'Romance'),
]

for palabra1, palabra2, descripcion in conceptos_principales:
    if palabra1 in modelo_w2v.wv.key_to_index and palabra2 in modelo_w2v.wv.key_to_index:
        try:
            similitud = modelo_w2v.wv.similarity(palabra1, palabra2)
            print(f"{descripcion} ({palabra1} - {palabra2}): {similitud:.3f}")
        except:
            continue

=== ANÁLISIS SEMÁNTICO DE CONCEPTOS CLAVE ===


--- AMOR ROMANCE ---
Palabras disponibles: ['love', 'woman', 'baby', 'heart', 'sweet']
  Similares a 'love': ['land', 'worry', 'bring', 'smiled', 'sayin']
  Similares a 'woman': ['sister', 'listen', 'free', 'smile', 'call']

--- ESPIRITUALIDAD ---
Palabras disponibles: ['jah', 'lord', 'soul', 'god']
  Similares a 'jah': ['remember', 'note', 'pass', 'singing', 'mount']
  Similares a 'lord': ['bring', 'fadeout', 'listen', 'school', 'note']

--- LUCHA SOCIAL ---
Palabras disponibles: ['fight', 'freedom', 'rights', 'rebel', 'stand']
  Similares a 'fight': ['note', 'listen', 'pass', 'bring', 'put']
  Similares a 'freedom': ['note', 'hear', 'believe', 'pass', 'smile']

--- EMOCIONES ---
Palabras disponibles: ['feel', 'happy', 'cry', 'worry', 'smile']
  Similares a 'feel': ['note', 'listen', 'put', 'bring', 'doo']
  Similares a 'happy': ['birds', 'hear', 'note', 'shed', 'put']

--- MUSICA ---
Palabras disponibles: ['jammin', 'music', 'rock', 'si

## Conclusiones Finales

Síntesis del análisis de embeddings Word2Vec aplicado a las letras de Bob Marley.

### Conclusiones del Análisis

El análisis de embeddings Word2Vec aplicado a las letras de Bob Marley revela patrones lingüísticos y temáticos significativos:

#### **1. Limitaciones del Corpus Pequeño**

El tamaño reducido del corpus presenta desafíos importantes:
- **Baja varianza explicada por PCA (~25%)**: La reducción a 2 dimensiones pierde gran parte de la información semántica contenida en los vectores de 200 dimensiones.
- **Similitudes altas generalizadas**: Incluso términos conceptualmente diferentes muestran similitudes de 0.2-0.4, reflejando más co-ocurrencia textual que relaciones semánticas profundas.
- **Predominancia de patrones locales**: Los embeddings capturan principalmente qué palabras aparecen juntas en versos, más que relaciones semánticas abstractas.

#### **2. Identificación de Temas**

A pesar de las limitaciones, el modelo identifica correctamente los **tres pilares temáticos** de Bob Marley:

- **Espiritualidad Rastafari**: Términos como "jah", "soul", "africa", "rights" se agrupan consistentemente, reflejando la filosofía rastafari.
- **Amor y Romance**: "Love", "woman", "baby", "heart" forman clusters cohesivos, representando las caniones románticas.

- **Lucha**: "Fight", "freedom", "rebel", "stand" se relacionan semánticamente, evidenciando el discurso político en las letras.


Este trabajo demuestra tanto el **potencial** como las **limitaciones** de Word2Vec en corpus especializados. Aunque el tamaño reducido limita la profundidad semántica, el modelo logra identificar patrones culturales y temáticos significativos, validando su utilidad para análisis exploratorios de textos con vocabulario cohesivo y temáticamente homogéneo como las letras de un artista específico.
